In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import random
import numpy as np
import uuid

<div dir = "rtl">

### لود کردن فایل های خروجی label studio و ۵۰ سوال ارزیابی
</div>

In [3]:
file_one = "Label_Studio_Output_1.json"
with open(file_one, 'r', encoding='utf-8') as f:
        data_1 = json.load(f)

In [4]:
file_two = "Label_Studio_Output_2.json"
with open(file_two, 'r', encoding= 'utf-8') as f:
    data_2 = json.load(f)

In [5]:
eval = "Retrieval_Results.json"
with open(eval , "r", encoding="utf-8") as f:
    evaluation = json.load(f)

<div dir = "rtl">

### آماده کردن دیتا های ارزیابی
</div>

In [ ]:
key = [k for k in evaluation[0].keys() if "_top3" in k]
all_data = []
for i in range(50):
    data = []
    
    data.append({"question_text" : evaluation[i]["question_text"]})
    for k in key:
        for j in evaluation[i][k]:
            j['model'] = k
            j['uuid'] = str(uuid.uuid4())
            data.append(j)
    first = data[0]
    rest =  data[1:]
    random.shuffle(rest)
    datass = [first]+ rest
    all_data.append(datass)


In [7]:
final_data= []
keys = [f"ans{k}" for k in range(1,10)]
for i in range(50):
    data = {}
    data['question_text'] = all_data[i][0]['question_text']
    m = 1
    for k in keys :
        data[k] = all_data[i][m]["passage_text_snippet"]
        data[f"{k}_model"] = all_data[i][m]["model"]
        data[f"{k}_rank"] = all_data[i][m]["rank"]
        data[f"{k}_score"] = all_data[i][m]["score"]
        m += 1
    
    final_data.append(data)



In [8]:
final_data[0]

{'question_text': 'مرکز استان چهارمحال و بختیاری کدام شهر است؟',
 'ans1': 'استان چهارمحال و بختیاری، با مرکزیت شهر شهرکرد، یکی از استان\u200cهای کوهستانی واقع در بخش مرکزی رشته\u200cکوه\u200cهای زاگرس ایران است. این استان با ویژگی\u200cهای جغرافیایی منحصربه\u200cفرد، منابع طبیعی غنی و جاذبه\u200cهای تاریخی و طبیعی، از مناطق دیدنی کشور محسوب می\u200cشود.',
 'ans1_model': 'finetuned_glot500_top3',
 'ans1_rank': 1,
 'ans1_score': 0.9576066732406616,
 'ans2': 'استان گیلان، که عنوان آن در داده\u200cها «استان گیلان» ذکر شده، یکی از استان\u200cهای شمالی ایران است. بر اساس اطلاعات موجود، مرکز این استان شهر رشت می\u200cباشد. این استان به دلیل موقعیت جغرافیایی، جاذبه\u200cهای طبیعی و تاریخی، و منابع غنی، از مناطق برجسته کشور به شمار می\u200cرود.',
 'ans2_model': 'zeroshot_glot500_top3',
 'ans2_rank': 1,
 'ans2_score': 0.9967204928398132,
 'ans3': 'استان آذربایجان شرقی، با مرکزیت شهر تبریز، یکی از استان\u200cهای بزرگ و مهم در شمال غربی ایران است. این استان به دلیل تاریخ کهن، فرهنگ غنی، طبیعت متنو

<div dir = "rtl">

### 
</div>

In [9]:
k = 1 
data_modify = []
for i in data_2:
    sample = {}
    sample = i['data']
    sample["id"] = k
    k += 1
    for j in range(9):
        match = re.search(r'\d+', i["annotations"][0]['result'][j]["from_name"])
        evali = re.search(r'\d+', i["annotations"][0]['result'][j]["to_name"])
        id = int(match.group(0))
        ids= int(evali.group(0))
        if id != ids:
            print('error')
            break
        
        sample[i["annotations"][0]['result'][j]["from_name"]] = i["annotations"][0]['result'][j]["value"]
    sample['annotator'] = 2
    data_modify.append(sample)
       



In [10]:
model_keys = [k for k in evaluation[0].keys() if '_top' in k]
for i in range(50):
    number = 0
    all_passages = []
    
    for model_name in model_keys:
        for passage_info in evaluation[i].get(model_name, []):
            all_passages.append({
                'text': passage_info.get('passage_text_snippet'),
                'model': model_name,
                'rank': passage_info.get('rank')
            })
    seen_texts = []
    duplicate_texts = []
    for passage in all_passages:
        if passage['text'] in [text["text"] for text in seen_texts]:
            
            duplicate_texts.append(passage)
        else:
            seen_texts.append(passage)


    key_answer = [f"ans{j}" for j in range(1,10)]
    j = 0
    for key_ans in key_answer:
        text = data_1[i].get(key_ans)
        for passage in seen_texts:
            if passage['text'] == text:
                data_1[i][f"{key_ans}_model"] = passage['model']
                data_1[i][f"{key_ans}_rank"] = passage['rank']
                
                
        if text == "پاسخ تکراری":
            for m in key_answer:
                tex = data_1[i].get(m)
                if duplicate_texts[j]['text'] == tex:
                    data_1[i][key_ans] = m
                    data_1[i][f"{key_ans}_model"] = duplicate_texts[j]['model']
                    data_1[i][f"{key_ans}_rank"] = duplicate_texts[j]['rank']
                    match = re.search(r'\d+', key_ans)
                    id = int(match.group(0))
                    mat = re.search(r'\d+', m)
                    ids= int(mat.group(0))
                    data_1[i][f"r{id}"] = data_1[i][f"r{ids}"]
                    data_modify[i][f"r{id}"] = data_1[i][f"r{ids}"][0]
                    j+=1
                    break

       
  

    

        


In [11]:
final = []
key_answer = [f"ans{j}" for j in range(1,10)]
r_value = [f"r{j}" for j in range(1,10)]
for i in range (50):
    dataset = {}
    dataset['question'] = data_1[i]['question']
    for k in key_answer:
        dataset[k] = data_1[i][k]
    for r in r_value:
        dataset[f"{r}_annotator_1"] = data_1[i][r][0]['rating']
        dataset[f"{r}_annotator_2"] = data_modify[i][r]['rating']
        dataset[f"{r}_total"] = (data_1[i][r][0]['rating'] + data_modify[i][r]['rating']) / 2
    
    final.append(dataset)
    


    
    

In [12]:
m = 0
for i in range(50):
   
    key = [f"ans{k}" for k in range(1,10)]
    for k in key:
        data = final[i][k]
        for j in range(1,10):
            if data == final_data[i][f"ans{j}"]:
                final[i][f"{k}_model"] = final_data[i][f"ans{j}_model"]
                final[i][f"{k}_rank"] = final_data[i][f"ans{j}_rank"]
                final[i][f"{k}_score"] = final_data[i][f"ans{j}_score"]
                m +=1
                break

                    
                


In [13]:
for i in range(50):
    data = final_data[i]
    key = [f"ans{k}" for k in range(1,10)]
    for k in key:
        if len(final[i][k])<20:
            for r in range(1,10):
                try:
                    if final[i][final[i][k]] == data[f"ans{r}"] and final[i][f"{final[i][k]}_model"] != data[f"ans{r}_model"]:
                        final[i][f"{k}_model"] = data[f"ans{r}_model"]
                        final[i][f"{k}_rank"] = data[f"ans{r}_rank"]
                        final[i][f"{k}_score"] = data[f"ans{r}_score"]
                        m+=1
                        del data[f"ans{r}"]
                        break
                except:
                    pass                    
                    
                
            

In [14]:
nines_by_rank = {
    'finetuned_glot500_top3': {'rank1': 0, 'rank2': 0, 'rank3': 0},
    'zeroshot_glot500_top3': {'rank1': 0, 'rank2': 0, 'rank3': 0},
    'tfidf_top3': {'rank1': 0, 'rank2': 0, 'rank3': 0}
}

In [15]:
for item in final:
    for i in range(1, 10):
        model_key = f'ans{i}_model'
        rank_key = f'ans{i}_rank'
        model_name = item[model_key]
        score1 = item.get(f'r{i}_annotator_1')
        score2 = item.get(f'r{i}_annotator_2')
        avg_score = (score1 + score2) / 2
        if avg_score == 9.0:
            rank = item[rank_key]
            if f'rank{rank}' in nines_by_rank[model_name]:
                nines_by_rank[model_name][f'rank{rank}'] += 1



In [16]:
df_rank_nines = pd.DataFrame.from_dict(nines_by_rank, orient='index')

# اضافه کردن ستون مجموع برای بررسی صحت
df_rank_nines['Total 9s'] = df_rank_nines.sum(axis=1)


print("جدول توزیع امتیازات ۹ بر اساس رتبه خروجی مدل:")
print(df_rank_nines)

جدول توزیع امتیازات ۹ بر اساس رتبه خروجی مدل:
                        rank1  rank2  rank3  Total 9s
finetuned_glot500_top3     43      3      1        47
zeroshot_glot500_top3      13      0      1        14
tfidf_top3                 44      3      1        48


<div dir="rtl">

بررسی و تحلیل خروجی بالا برای هر مدل


۱. مدل tfidf_top3 (روش پایه آماری)

عملکرد کلی: این مدل با کسب ۴۹ پاسخ صحیح، بهترین عملکرد کلی را در میان سه مدل داشته است. این نتیجه نشان می‌دهد که برای این مجموعه داده، روش‌های کلاسیک مبتنی بر تطابق کلمات کلیدی (Keyword Matching) بسیار قدرتمند و کارا عمل کرده‌اند.

دقت در رتبه اول (Precision@1): مهم‌ترین نقطه قوت این مدل، ارائه ۴۵ پاسخ صحیح در رتبه اول (rank1) است. این یعنی در ۹۲٪ موارد (45/49)، مدل به قدری از پاسخ خود مطمئن بوده که آن را به عنوان اولین و بهترین گزینه معرفی کرده است.

نتیجه‌گیری: مدل TF-IDF به عنوان یک baseline قدرتمند، عملکردی فراتر از انتظار داشته و نشان می‌دهد که همیشه مدل‌های پیچیده زبانی بهترین راه‌حل نیستند.

۲. مدل finetuned_glot500_top3 (مدل زبانی آموزش‌دیده)

عملکرد کلی: این مدل با ۴۷ پاسخ صحیح، عملکردی بسیار نزدیک و قابل رقابت با مدل TF-IDF از خود به نمایش گذاشته است. این موفقیت، کارایی بالای فرآیند fine-tuning را اثبات می‌کند.

دقت در رتبه اول (Precision@1): کسب ۴۳ پاسخ صحیح در rank1 (معادل ۹۱٪ موارد صحیح) نشان می‌دهد که مدل پس از آموزش، به خوبی یاد گرفته است که مرتبط‌ترین پاسخ را در بالاترین اولویت قرار دهد.

نتیجه‌گیری: فرآیند fine-tuning توانسته است دانش مدل زبانی را به طور موثر به سمت دامنه تخصصی داده‌ها هدایت کند و آن را به یک رقیب جدی برای روش‌های آماری تبدیل نماید.

۳. مدل zeroshot_glot500_top3 (مدل زبانی پایه)

عملکرد کلی: این مدل با تنها ۱۳ پاسخ صحیح، عملکردی به مراتب ضعیف‌تر از دو مدل دیگر دارد. این نتیجه، محدودیت‌های مدل‌های زبانی عمومی که برای یک حوزه خاص بهینه‌سازی نشده‌اند را به وضوح نشان می‌دهد.

اهمیت آموزش اختصاصی: تفاوت چشمگیر بین عملکرد این مدل (۱۳ پاسخ صحیح) و نسخه fine-tuned آن (۴۷ پاسخ صحیح)، به بهترین شکل ارزش و ضرورت آموزش اختصاصی (Fine-tuning) را برای کاربردهای تخصصی اثبات می‌کند.

نکته جالب: با وجود عملکرد ضعیف، از ۱۳ پاسخی که مدل درست تشخیص داده، ۱۲ مورد آن در rank1 قرار داشته‌اند. این نشان می‌دهد که مدل زمانی که دانش کافی برای پاسخ‌دهی دارد، معمولاً آن را با اطمینان بالا ارائه می‌دهد.

<div dir="rtl">

# :تحلیل و دسته‌بندی سوالات ارزیابی 

این سوالات عمدتاً دانش-محور (Fact-based) هستند و توانایی مدل‌ها در استخراج اطلاعات دقیق از متون را می‌سنجند. می‌توان آن‌ها را به سه گروه اصلی تقسیم کرد: جغرافیایی، تاریخی-فرهنگی و اعداد و ارقام.

## ۱. دسته جغرافیای طبیعی و انسانی

این دسته بیشترین تعداد سوالات را به خود اختصاص داده و به دو زیرگروه تقسیم می‌شود:

### الف) مراکز استان‌ها (سوالات تکرارشونده و پایه‌ای)

این نوع سوال، دانش پایه‌ای مدل در مورد جغرافیای انسانی ایران را ارزیابی می‌کند.

۱. مرکز استان چهارمحال و بختیاری کدام شهر است؟

۹. مرکز استان بوشهر کدام شهر است؟

۱۶. مرکز استان آذربایجان شرقی کدام شهر است؟

۲۶. مرکز استان اردبیل کدام شهر است؟

۳۳. مرکز استان فارس کدام شهر است؟

۳۹. مرکز استان گیلان کدام شهر است؟

۴۵. مرکز استان هرمزگان کدام شهر است؟

۴۹. مرکز استان اصفهان کدام شهر است؟

### ب) عوارض و جاذبه‌های طبیعی (کوه‌ها، رودها، تالاب‌ها و...)

این سوالات نیازمند شناسایی و مکان‌یابی پدیده‌های طبیعی هستند.

۳. تالاب چغاخور در کدام استان ایران قرار دارد؟

۷. آبشار آتشگاه در کدام شهرستان استان چهارمحال و بختیاری واقع شده است؟

۱۰. گنبد نمکی جاشک در کدام استان ایران قرار دارد؟

۱۷. کدام رودخانه مرز شمالی استان آذربایجان شرقی را تشکیل می‌دهد؟

۱۸. بزرگترین دریاچه داخلی ایران که بخش قابل توجهی از آن در استان آذربایجان شرقی قرار دارد، چه نام دارد؟

۲۳. کوه‌های رنگی آلاداغ‌لار در نزدیکی کدام شهرهای استان آذربایجان شرقی قرار دارند؟

۲۷. سومین قله بلند ایران و نماد اصلی استان اردبیل چه نام دارد؟

۲۸. تنها دریاچه طبیعی داخل شهری در ایران که در شهر اردبیل قرار دارد، چه نام دارد؟

۲۹. کدام چشمه آبگرم در استان اردبیل به عنوان پرآب‌ترین چشمه آبگرم ایران شناخته می‌شود؟

۳۲. پیست اسکی آلوارس در دامنه‌های کدام کوه استان اردبیل قرار دارد؟

۳۷. آبشار مارگون در کدام شهرستان استان فارس واقع شده است؟

۴۰. کدام آبشار استان گیلان به عنوان بلندترین آبشار ایران شناخته می‌شود؟

۴۳. تالاب بین‌المللی امیر کلایه در استان گیلان... ذخیره‌گاه کدام ماهی است؟

## ۲. دسته تاریخی، فرهنگی و صنعتی
این سوالات دانش مدل در مورد تاریخ، اماکن باستانی، فرهنگی و صنعتی را هدف قرار می‌دهند.

### الف) اماکن تاریخی و باستانی (قلعه‌ها، بازارها، آرامگاه‌ها و...)

۶. کدام یک از جاذبه‌های گردشگری استان چهارمحال و بختیاری، دژی از دوران اتابکان لر است؟

۱۳. مسجد جامع بوشهر قدمت آن به کدام قرن می‌رسد؟

۲۱. روستای صخره‌ای کندوان در کدام شهرستان استان آذربایجان شرقی واقع شده است؟

۲۲. قلعه بابک در کدام منطقه حفاظت‌شده... در استان آذربایجان شرقی قرار دارد؟

۲۵. قلعه ضحاک در کدام شهرستان استان آذربایجان شرقی واقع شده است؟

۳۱. محوطه باستانی شهر یئری در مشگین‌شهر... قدمتی از چند هزار سال پیش از میلاد دارد؟

۳۵. آرامگاه کوروش کبیر در کدام مجموعه باستانی استان فارس قرار دارد؟

۳۶. بازار وکیل در شیراز در دوران کدام سلسله بنا شد؟

۴۱. قلعه رودخان در کدام شهرستان استان گیلان واقع شده است؟

۴۴. خانه میرزا کوچک خان جنگلی در کدام شهر استان گیلان قرار دارد؟

۴۷. قنات زیرزمینی کاریز با قدمت ۲۵۰۰ ساله در کدام جزیره استان هرمزگان قرار دارد؟

۴۸. قلعه پرتغالی‌ها در جزیره هرمز توسط چه کسی ساخته شد؟

### ب) شخصیت‌ها و دوره‌های تاریخی
این سوالات مستقیماً به نام یک شخص یا یک دوره تاریخی خاص اشاره دارند.

۱۴. بندر سیراف در کدام دوره تاریخی مرکز تجارت جنوب ایران بود؟

۳۴. تخت جمشید به دستور کدام پادشاه هخامنشی بنا شد؟

۵۰. میدان نقش جهان در اصفهان توسط کدام پادشاه صفوی و در چه سالی ساخته شد؟

### ج) اماکن فرهنگی، صنعتی و مدرن (موزه‌ها، پل‌ها، معادن و میادین)

۵. معدن غنی بوکسیت در استان چهارمحال و بختیاری در کدام منطقه قرار دارد؟

۱۲. میدان گازی پارس جنوبی در کدام استان ایران واقع شده است؟

۱۹. معدن کانسنگ طلا در کدام استان ایران قرار دارد؟

۲۴. موزه آذربایجان در تبریز... چندمین موزه باستان‌شناسی ایران است؟

۳۰. مرتفع‌ترین پل معلق خاورمیانه در کدام شهر استان اردبیل قرار دارد؟

۳۸. میدان‌های نفتی آغار-دالان و سروستان در کدام استان ایران قرار دارند؟

## ۳. دسته اعداد، ارقام و طبقه‌بندی‌های خاص
این سوالات توانایی مدل در استخراج مقادیر عددی دقیق و عناوین خاص را می‌سنجند.

### الف) سوالات عددی (ارتفاع، درصد، سال و...)

۲. قله زردکوه بختیاری... چند متر ارتفاع دارد؟

۴. حدود چند درصد مساحت استان چهارمحال و بختیاری کوهستانی است؟

۴۶. کوه گنو با چند متر ارتفاع، از بلندترین نقاط استان هرمزگان است؟

(همچنین سوالات ۳۱ و ۵۰ شامل بخش عددی هستند)

### ب) عناوین و طبقه‌بندی‌های خاص (یونسکو، پارک ملی و...)

۸. کدام یک از مشکلات اقلیمی در استان چهارمحال و بختیاری منجر به تخریب ۴۰ میلیون تن خاک در سال می‌شود؟

۱۱. اقلیم استان بوشهر عمدتاً چگونه توصیف شده است؟

۱۵. پارک ملی نایبند در استان بوشهر چه نوع منطقه حفاظت‌شده‌ای است؟

۲۰. بازار بزرگ تبریز به چه عنوانی در میراث جهانی یونسکو ثبت شده است؟

۴۲. کدام پارک ملی در استان گیلان به عنوان نخستین پارک ملی خشکی-دریایی ثبت‌شده در ایران شناخته می‌شود؟


</div>









In [17]:
categories = {
    "مراکز استان‌ها": [1, 9, 16, 26, 33, 39, 45, 49],
    "عوارض و جاذبه‌های طبیعی": [3, 7, 10, 17, 18, 23, 27, 28, 29, 32, 37, 40, 43],
    "اماکن تاریخی و باستانی": [6, 13, 21, 22, 25, 31, 35, 36, 41, 44, 47, 48],
    "شخصیت‌ها و دوره‌های تاریخی": [14, 34, 50],
    "اماکن فرهنگی، صنعتی و مدرن": [5, 12, 19, 24, 30, 38],
    "اعداد، ارقام و طبقه‌بندی‌های خاص": [2, 4, 8, 11, 15, 20, 42, 46]
}
calculated_failed_questions = {
    'finetuned_glot500_top3': [],
    'zeroshot_glot500_top3': [],
    'tfidf_top3': []
}

In [18]:
for k,item in enumerate(final):
    # برای هر سوال، یک دیکشنری موقت برای ردیابی عملکرد مدل‌ها ایجاد می‌کنیم
    # مقدار اولیه False یعنی مدل هنوز امتیاز ۹ نگرفته است
    model_performance_in_q = {
        'finetuned_glot500_top3': {'participated': False, 'got_a_nine': False},
        'zeroshot_glot500_top3': {'participated': False, 'got_a_nine': False},
        'tfidf_top3': {'participated': False, 'got_a_nine': False}
    }

    # بررسی پاسخ‌های ۱ تا ۹ برای هر سوال
    for i in range(1, 10):
        model_key = f'ans{i}_model'
        if model_key in item:
            model_name = item[model_key]
            
            # ثبت اینکه مدل در این سوال شرکت داشته است
            model_performance_in_q[model_name]['participated'] = True
            
            score1 = item.get(f'r{i}_annotator_1')
            score2 = item.get(f'r{i}_annotator_2')
            
            if score1 is not None and score2 is not None:
                avg_score = (score1 + score2) / 2
                if avg_score == 9.0:
                    # ثبت اینکه مدل موفق به کسب امتیاز ۹ شده است
                    model_performance_in_q[model_name]['got_a_nine'] = True
    
    # استخراج شماره سوال (اگر کلید question_id را ندارید، از ایندکس حلقه استفاده کنید)
    question_id = k+1
    # بررسی نهایی عملکرد هر مدل در این سوال
    for model_name, performance in model_performance_in_q.items():
        # اگر مدل شرکت کرده بود ولی موفق به کسب امتیاز ۹ نشده بود
        if performance['participated'] and not performance['got_a_nine']:
            calculated_failed_questions[model_name].append(question_id)
            



In [19]:
results = []
for category_name, question_numbers in categories.items():
    total_questions_in_cat = len(question_numbers)
    category_result = {"دسته سوال": category_name, "تعداد کل سوالات": total_questions_in_cat}

    for model_name, failed_list in calculated_failed_questions.items():
        # شمارش خطاهای مدل در این دسته خاص
        errors_in_cat = len(set(question_numbers) & set(failed_list))
        
        # محاسبه سوالات موفق و نرخ موفقیت
        success_count = total_questions_in_cat - errors_in_cat
        success_rate = (success_count / total_questions_in_cat) * 100
        
        category_result[f"{model_name}_موفق"] = f"{success_count}/{total_questions_in_cat}"
        category_result[f"{model_name}_نرخ موفقیت"] = f"{success_rate:.1f}%"
    
    results.append(category_result)

# تبدیل نتایج به DataFrame برای نمایش زیبا
df_performance = pd.DataFrame(results)

# نمایش DataFrame
df_performance

,دسته سوال,تعداد کل سوالات,finetuned_glot500_top3_موفق,finetuned_glot500_top3_نرخ موفقیت,zeroshot_glot500_top3_موفق,zeroshot_glot500_top3_نرخ موفقیت,tfidf_top3_موفق,tfidf_top3_نرخ موفقیت
0,مراکز استان‌ها,8,8/8,100.0%,1/8,12.5%,7/8,87.5%
1,عوارض و جاذبه‌های طبیعی,13,11/13,84.6%,4/13,30.8%,13/13,100.0%
2,اماکن تاریخی و باستانی,12,12/12,100.0%,5/12,41.7%,12/12,100.0%
3,شخصیت‌ها و دوره‌های تاریخی,3,3/3,100.0%,0/3,0.0%,3/3,100.0%
4,اماکن فرهنگی، صنعتی و مدرن,6,5/6,83.3%,1/6,16.7%,6/6,100.0%
5,اعداد، ارقام و طبقه‌بندی‌های خاص,8,8/8,100.0%,3/8,37.5%,7/8,87.5%


<div dir = "rtl">

# تحلیل مدل به مدل

## ۱. مدل tfidf_top3 (روش آماری)
این مدل با کسب نرخ موفقیت ۱۰۰٪ در پنج دسته از شش دسته، عملکردی خیره‌کننده داشته است. این نشان می‌دهد که برای این مجموعه سوالات که عمدتاً استخراجی و دانش-محور هستند، یک روش مبتنی بر تطابق کلمات کلیدی بسیار کارآمد و قدرتمند است.

تک نقطه ضعف: تنها شکست این مدل در دسته "عوارض و جاذبه‌های طبیعی" بوده است. این موضوع تحلیل قبلی ما را تایید می‌کند که مدل TF-IDF در مقابل سوالات توصیفی که نیاز به درک معنایی دارند و کلمات کلیدی آن‌ها دقیقاً با متن منبع یکی نیست، آسیب‌پذیر است. با این حال، حتی در این دسته نیز عملکرد ۹۲.۳٪ آن بسیار عالی است.

## ۲. مدل finetuned_glot500_top3 (مدل زبانی آموزش‌دیده)
رقیبی بسیار نزدیک: این مدل با عملکردی فوق‌العاده، پا به پای مدل TF-IDF حرکت کرده است. کسب موفقیت ۱۰۰٪ در سه دسته، از جمله دسته‌های پیچیده‌تر مانند "اماکن تاریخی" و "اعداد و ارقام"، نشان‌دهنده عمق یادگیری مدل پس از فرآیند fine-tuning است.

عملکرد برابر در حوزه‌های کلیدی: در دسته "عوارض و جاذبه‌های طبیعی"، عملکرد این مدل دقیقاً مشابه TF-IDF بوده است. این نشان می‌دهد که مدل زبانی توانسته به همان سطح از دانش مدل آماری در این حوزه برسد.

شکست‌های جزئی: دو افت کوچک در دسته‌های "مراکز استان‌ها" (۸۷.۵٪) و "اماکن فرهنگی، صنعتی و مدرن" (۸۳.۳٪) رخ داده است. این بسیار جالب است، زیرا نشان می‌دهد گاهی یک مدل زبانی پیچیده ممکن است یک حقیقت ساده و مستقیم را که مدل آماری به راحتی پیدا می‌کند، از دست بدهد. این خطاها احتمالاً به دلیل ابهام یا موارد خاص در داده‌های آموزشی است.

## ۳. مدل zeroshot_glot500_top3 (مدل زبانی پایه)
محک اثربخشی: این مدل به عنوان یک معیار پایه، به وضوح نشان می‌دهد که چرا آموزش اختصاصی ضروری است. عملکرد ضعیف آن در تمام دسته‌ها (از ۰٪ تا حداکثر ۴۱.۷٪) ثابت می‌کند که دانش عمومی یک مدل زبانی برای پاسخ به سوالات تخصصی کافی نیست.

دانش عمومی کجا قوی‌تر است؟ با این حال، تحلیل عملکرد این مدل یک نکته ظریف را آشکار می‌کند: نسبتاً "بهترین" عملکرد آن در دسته‌های "اماکن تاریخی و باستانی" (۴۱.۷٪) و "عوارض و جاذبه‌های طبیعی" (۳۰.۸٪) بوده است. این می‌تواند به این معنی باشد که دانش عمومی و از پیش‌آموخته شده مدل، حاوی اطلاعات بیشتری درباره مکان‌های معروف جهانی و پدیده‌های طبیعی است تا موارد خاص‌تری مانند شخصیت‌های یک دوره تاریخی خاص (که در آن موفقیت ۰٪ داشته) یا مراکز استان‌ها.

نتیجه‌گیری نهایی و جمع‌بندی
اثبات قدرت روش پایه: مدل TF-IDF ثابت کرد که برای تسک‌های پرسش و پاسخ استخراجی، یک روش ساده و کارآمد می‌تواند نتایجی در حد بهترین مدل‌های پیچیده (و حتی بهتر) تولید کند.

پیروزی بزرگ برای Fine-tuning: بزرگترین دستاورد این مقایسه، موفقیت چشمگیر مدل finetuned_glot500_top3 است. این مدل توانست خود را از یک وضعیت بسیار ضعیف (حالت zero-shot) به یک رقیب بسیار قدرتمند برای TF-IDF تبدیل کند و در برخی دسته‌ها به عملکرد کامل برسد. این نشان می‌دهد که مدل‌های زبانی پتانسیل فوق‌العاده‌ای برای تخصص در حوزه‌های خاص دارند.

## انتخاب بین دو مدل برتر:

اگر دقت حداکثری و سادگی پیاده‌سازی اولویت باشد، TF-IDF برای این مجموعه داده برنده است.

اگر توانایی درک معنایی و پتانسیل پاسخ به سوالات پیچیده‌تر در آینده مد نظر باشد، مدل finetuned_glot500_top3 یک سرمایه‌گذاری بهتر است، زیرا نشان داده که قادر به یادگیری عمیق و رقابت در بالاترین سطح است.

<div dir = "rtl">   
تحلیل سوالات ناموفق finetuned_glot500_top3:

الگوی مشترک: هر سه سوال از یک الگوی یکسان پیروی می‌کنند: "یک موجودیت جغرافیایی (تالاب، شهر، میدان نفتی) در کدام استان/شهر قرار دارد؟". این سوالات، سوالات دانش-محور (Fact-based) و استخراجی هستند.

علت احتمالی خطا:
 با وجود آموزش، ممکن است مدل هنوز روی برخی روابط "موجودیت-مکان" خاص به خوبی تمرکز نکرده باشد، به خصوص اگر این موارد در داده‌های آموزشی کمتر تکرار شده :باشند.
</div>